In [1]:
import openai
from langchain.document_loaders import PyPDFLoader
import json

In [2]:
# OpenAI API 키 설정
openai.api_key = "개인키입력"

In [3]:
def text_generator(prompt, model="gpt-4o", temperature=0):
    messages = [
        {"role": "system", "content": "질문에 오류 혹은 잘못된 정보가 있는지 확인하고, 있다며 이것을 지적하고 수정한다."},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
    )
    answer = response.choices[0].message.content
    return answer

In [4]:
def get_arg_answer(query, verbose=False):
    prompt = f"""
    너는 포트미스 가이드북에 대해 잘 알고 있는 전문가야. 
    주어진 텍스트를 기반으로 추가적인 질문 100개를 생성해줘.
    
    문장의 구분은 ' / '로 진행해줘.
    문장들을 구분하는데 있어서 숫자나 ' / '를 제외한 기호가 있다면 제외하고 출력해.

    포트미스 가이드북은 다음과 같은 내용으로 구성되어 있어. :
        포트미스 주요 서비스/이용 환경/회원가입 절차, 
        입·출항 선박에 대한 선박제원 등록,
        (변경) 절차 업무 흐름,
        외항선 민원신고 절차,
        내항선 민원신고 절차,
        외항선입항(출항)신고(최초,변경),
        승무원/승객명부,
        강제도선 면제신청,
        예선사용 면제신청,
        선박평형수신고,
        선박보안정보통보신고,
        저속운항선박지원신청,
        저속운항선박 인센티브 신청,
        항만시설사용허가신청서(선석신청),
        항만시설사용료(정박료)면제신청,
        위험물 반입신고 / 일람표신고,
        통합화물신고,
        외항입항(출항)신고(최종),
        항만시설사용신고(화물료),
        화물료 대납경비 청구,
        국가관리연안항신고 및 연안항 화물료 신고,
        서북도서출항신고,
        월정료선박사용신청,
        선박수리신고 신청 / 선박수리허가 신청,
        상계/환급 신청,
        포트미스 히스토리,
        부록 신고서식,
        항만시설 사용료 디지털 방식 

    
    ```
    exemple
    input : 포트미스 가이드북
    output : 포트미스의 주요 서비스는 무엇인가요? / 포트미스 이용 환경은 어떻게 구성되어 있나요? / 포트미스 회원가입 절차는 어떻게 되나요? / 입·출항 선박에 대한 선박제원 등록 절차는 무엇인가요? / 외항선 민원신고 절차는 어떻게 되나요? / 내항선 민원신고 절차는 어떻게 되나요? / 외항선 입항신고는 어떻게 진행되나요? / 강제도선 면제신청은 어떻게 이루어지나요?  
    ```
    
    그리고, "포트미스 가이드북에 대한 추가적인 질문을 생성해드리겠습니다." 이런 설명은 빼고 그냥 질문만 작성해줘.
    문장들을 구분하는데 있어서 "1."와 같이 숫자가 있다면 그것도 모두 제외해줘.
    ex) 1. 포트미스의 주요 서비스는 무엇인가요? -> 포트미스의 주요 서비스는 무엇인가요?
    
    자 이제 너가 생성해야할 글을 제공해줄게
    
    {query}
    """
    if verbose:
        print(f'Prompt: {prompt}')
    answer = text_generator(prompt)
    return answer

In [5]:
def get_gen_answer(questions, verbose=False):
    questions_list = questions.split(" / ")
    answers = []
    for question in questions_list:
        prompt = f"""
        너는 포트미스 가이드북에 대해 잘 알고 있는 전문가야. 
        주어진 질문에 대해 전문가로서 답변을 해줘.
        가능한 한 관련 규정과 사전 지식을 최대한 활용하여 답변을 출력해줘.
        빠르게 답변할 필요 없이 생각하고 답변을 출력해줘.

        출력물의 최대 문자 길이는 300자로 제한할게.
        답변을 최대 문자 길이 내로 축약해서 전달해줘.

        답변은 예시와 같이 제공하면 돼.
        ```
        input : 포트미스의 주요 서비스는 무엇인가요? / 포트미스 이용 환경은 어떻게 구성되어 있나요? / 포트미스 회원가입 절차는 어떻게 되나요? / 입·출항 선박에 대한 선박제원 등록 절차는 무엇인가요? / 외항선 민원신고 절차는 어떻게 되나요? / 내항선 민원신고 절차는 어떻게 되나요? / 외항선 입항신고는 어떻게 진행되나요? / 강제도선 면제신청은 어떻게 이루어지나요?  
        output : 포트미스는 해운민원, 선박용물건, 승하선공인, 운임공표 등록, 선원업무, 해운사업자, 선박업무, 항만민원, 관제, 예도선, 사용료징수, 화물/위험물, 항만시설, 선박운항, 항만물류 통계, 컨테이너 통계, 화물통계, 선박입출항 통계, 수출입물류 통계 등 다양한 서비스를 제공합니다. /
                 포트미스는 무료로 제공되는 인터넷 웹 서비스와 유료로 제공되는 다양한 중계망 사업자를 통한 모바일 서비스로 구성되어 있으며, 민원 업무 신고 및 결과 조회, 항만 사용료 납부 및 조회, 통계 정보 제공 등의 기능을 제공합니다. /
                 포트미스 회원가입 절차는 회원가입 정보 작성, 사용 업체 등록 신청, 첨부서류 제출, 신청정보 확인, 신청정보 승인, 회원가입 완료로 이루어집니다. / 
                 포트미스 회원가입 후 선박제원 신청을 진행하며, 포트미스 접속, 선박제원 정보 작성, 첨부서류 제출, 신청정보 처리 현황 조회, 선박제원 신고 내용 확인, 신청정보 승인, 입항(출항) 신고 신청 등의 절차로 이루어집니다. / 
                 외항선 민원신고 절차는 포트미스 접속 후 신고서를 작성하고, 필요 서류를 제출한 뒤, 담당자의 검토와 승인을 거쳐 완료됩니다. / 
                 내항선 민원신고 절차는 외항선 민원신고 절차와 유사하게 포트미스 접속 후 신고서 작성, 서류 제출, 담당자 검토 및 승인으로 이루어집니다. / 
                 외항선 입항신고는 포트미스 접속 후 입항신고서를 작성하고, 필요한 서류를 제출한 뒤, 담당자의 확인 및 승인 절차를 거칩니다. / 
                 강제도선 면제신청은 1년 이내 4회 이상 또는 3년 이내 9회 이상 승선한 선장과 선박의 경우 신청할 수 있으며, 신청 후 면제증 발급, 입출항 신고시 면제증 제출 등의 절차로 이루어집니다.
        ```

        답변은 output의 내용만 포함하면 돼.
        꼭 아래와 같이 잘못된 형식은 고쳐줘야해!
        ex) ```\ninput : 포트미스 주요 서비스는 무엇인가요?\noutput : 포트미스는 해운민원, 선박용물건, 승하선공인, 운임공표 등록, 선원업무, 해운사업자, 선박업무, 항만민원, 관제, 예도선, 사용료징수, 화물/위험물, 항만시설, 선박운항, 항만물류 통계, 컨테이너 통계, 화물통계, 선박입출항 통계, 수출입물류 통계 등 다양한 서비스를 제공합니다.\n```
        -> 포트미스는 해운민원, 선박용물건, 승하선공인, 운임공표 등록, 선원업무, 해운사업자, 선박업무, 항만민원, 관제, 예도선, 사용료징수, 화물/위험물, 항만시설, 선박운항, 항만물류 통계, 컨테이너 통계, 화물통계, 선박입출항 통계, 수출입물류 통계 등 다양한 서비스를 제공합니다.
        
        
        자 이제 너가 답변해야할 글을 제공해줄게
        {question}
        """
        if verbose:
            print(f'Prompt for question "{question}": {prompt}')
        answer = text_generator(prompt)
        answers.append(answer)
    return answers

In [6]:
def generate_qa_from_pdf(pdf_path, verbose=False):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text = " ".join([page.page_content for page in docs])
    
    questions = get_arg_answer(text, verbose=verbose)
    answers = get_gen_answer(questions, verbose=verbose)

    return questions, answers

In [7]:
pdf_path = "D:/[24]ICT_Practice/practice_file/(2024)포트미스_가이드북.pdf"
questions, answers = generate_qa_from_pdf(pdf_path)

qa_pairs = [{"QUESTION": q, "ANSWER": a} for q, a in zip(questions.split(" / "), answers)]

print(json.dumps(qa_pairs, ensure_ascii=False, indent=2))

[
  {
    "QUESTION": "포트미스의 주요 서비스는 무엇인가요?",
    "ANSWER": "포트미스는 해운민원, 선박용물건, 승하선공인, 운임공표 등록, 선원업무, 해운사업자, 선박업무, 항만민원, 관제, 예도선, 사용료징수, 화물/위험물, 항만시설, 선박운항, 항만물류 통계, 컨테이너 통계, 화물통계, 선박입출항 통계, 수출입물류 통계 등 다양한 서비스를 제공합니다."
  },
  {
    "QUESTION": "포트미스 이용 환경은 어떻게 구성되어 있나요?",
    "ANSWER": "포트미스는 무료로 제공되는 인터넷 웹 서비스와 유료로 제공되는 다양한 중계망 사업자를 통한 모바일 서비스로 구성되어 있으며, 민원 업무 신고 및 결과 조회, 항만 사용료 납부 및 조회, 통계 정보 제공 등의 기능을 제공합니다."
  },
  {
    "QUESTION": "포트미스 회원가입 절차는 어떻게 되나요?",
    "ANSWER": "포트미스 회원가입 절차는 회원가입 정보 작성, 사용 업체 등록 신청, 첨부서류 제출, 신청정보 확인, 신청정보 승인, 회원가입 완료로 이루어집니다."
  },
  {
    "QUESTION": "입·출항 선박에 대한 선박제원 등록 절차는 무엇인가요?",
    "ANSWER": "포트미스 회원가입 후 선박제원 신청을 진행하며, 포트미스 접속, 선박제원 정보 작성, 첨부서류 제출, 신청정보 처리 현황 조회, 선박제원 신고 내용 확인, 신청정보 승인, 입항(출항) 신고 신청 등의 절차로 이루어집니다."
  },
  {
    "QUESTION": "외항선 민원신고 절차는 어떻게 되나요?",
    "ANSWER": "외항선 민원신고 절차는 포트미스 접속 후 신고서를 작성하고, 필요 서류를 제출한 뒤, 담당자의 검토와 승인을 거쳐 완료됩니다."
  },
  {
    "QUESTION": "내항선 민원신고 절차는 어떻게 되나요?",
    "ANSWER": "내항선 민원신고 절차는 외항선 민원신고 절차와 유사하게 포트

In [8]:
# JSON 파일 저장
output_path = "D:/[24]ICT_Practice/practice_file/guidebook_qa100.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=2)

print(f"QA data has been saved to {output_path}")

QA data has been saved to D:/[24]ICT_Practice/practice_file/guidebook_qa100.json
